1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use Dense Layers (also called fully connected layers), or DropOut.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [2]:
%%time
import keras
import math
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import SeparableConv2D,Conv2D, Dense, Input, add, Activation, AveragePooling2D, GlobalAveragePooling2D, Lambda, concatenate
from keras.initializers import he_normal
from keras.layers.merge import Concatenate
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.models import Model
from keras.optimizers import *
from keras.regularizers import *
from keras.utils import to_categorical

Using TensorFlow backend.


Wall time: 7.3 s


In [17]:
#(X_train,y_train),(X_test,y_test)=cifar10.load_data()

In [3]:
%%time
X_train = np.load('data.npz')['a']
X_test = np.load('data.npz')['b']
y_train = np.load('data.npz')['c']
y_test = np.load('data.npz')['d']

Wall time: 1.52 s


In [4]:
%%time
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

Wall time: 932 ms


In [5]:
def learning_scheduler(epoch):
    if epoch < 150:
        return 0.1
    if epoch < 225:
        return 0.01
    return 0.001

In [6]:
%%time
def conv(x, out_filters, k_size):
    return(SeparableConv2D(filters=out_filters,kernel_size=k_size,strides=(1,1), padding='same',kernel_initializer='he_normal', 
                  kernel_regularizer=l2(weight_decay),use_bias=False)(x))

def dense_layer(x):
    return(Dense(units=classes_num,
                    activation='softmax',
                     kernel_initializer='he_normal',
                     kernel_regularizer=l2(weight_decay))(x))

def bn_relu(x):
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    return(x)

def bottleneck_layer(x):
    channels = growth_rate * 4
    x = bn_relu(x)
    x = conv(x, channels, (1,1))
    x = bn_relu(x)
    x = conv(x, growth_rate, (3,3))
    return(x)

def single(x):
    x = bn_relu(x)
    x = conv(x, growth_rate, (3,3))
    return(x)

def transition(x, inchannels):
    outchannels = int(inchannels * compression)
    x = bn_relu(x)
    x = conv(x, outchannels, (1,1))
    x = AveragePooling2D((2,2), strides=(2, 2))(x)
    return(x, outchannels)

def dense_block(x,blocks,nchannels):
    concat = x
    for i in range(blocks):
        x = bottleneck_layer(concat)
        concat = concatenate([x,concat], axis=-1)
        nchannels += growth_rate
    return(concat, nchannels)
def densenet(img_input,classes_num,depth,growth_rate,compression):
    #nblocks = (depth - 4) // 6 
    nchannels = growth_rate * 2
    x = conv(img_input, nchannels, (3,3))
    x, nchannels = dense_block(x,nblocks,nchannels)
    x, nchannels = transition(x,nchannels)
    x, nchannels = dense_block(x,nblocks,nchannels)
    x, nchannels = transition(x,nchannels)
    x, nchannels = dense_block(x,nblocks,nchannels)
    x = bn_relu(x)
    x = GlobalAveragePooling2D()(x)
    x = dense_layer(x)
    return(Model(img_input,x))

Wall time: 0 ns


In [7]:
growth_rate= 12 
depth = 64
compression = 0.5
batch_size = 64 
nb_epochs = 300
iterations = 200
classes_num = 10
weight_decay = 1e-4

In [8]:

model = densenet(Input(shape=X_train.shape[1:]),10,depth,growth_rate,compression)
model.summary()

W0926 23:00:11.588988 16052 deprecation_wrapper.py:119] From C:\Users\patha\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4074: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 32, 32, 24)   99          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          separable_conv2d_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [10]:
tb_cb     = TensorBoard(log_dir='./densenet/', histogram_freq=0)
change_lr = LearningRateScheduler(learning_scheduler)
ckpt      = ModelCheckpoint('./ckpt.h5', save_best_only=False, mode='auto', period=10)
cbks      = [change_lr,tb_cb,ckpt]


In [11]:
datagen = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.125,height_shift_range=0.125,fill_mode='nearest',cval=0.)
datagen.fit(X_train)


In [12]:
sgd = SGD(lr=.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [13]:
%%time
model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        epochs=nb_epochs,
                        validation_data=(X_test, y_test),callbacks=cbks,steps_per_epoch=iterations,
                        workers=4)

W0926 23:01:04.521609 16052 deprecation_wrapper.py:119] From C:\Users\patha\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0926 23:01:04.534597 16052 deprecation_wrapper.py:119] From C:\Users\patha\Anaconda3\lib\site-packages\keras\callbacks\tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0926 23:01:04.535587 16052 deprecation_wrapper.py:119] From C:\Users\patha\Anaconda3\lib\site-packages\keras\callbacks\tensorboard_v1.py:203: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/300
200/200 [==============================] - 72s 361ms/step - loss: 1.9063 - accuracy: 0.2834 - val_loss: 1.8889 - val_accuracy: 0.3041


W0926 23:02:18.830979 16052 deprecation_wrapper.py:119] From C:\Users\patha\Anaconda3\lib\site-packages\keras\callbacks\tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/300
200/200 [==============================] - 62s 311ms/step - loss: 1.5894 - accuracy: 0.4134 - val_loss: 2.0820 - val_accuracy: 0.3291
Epoch 3/300
200/200 [==============================] - 62s 310ms/step - loss: 1.3914 - accuracy: 0.4960 - val_loss: 1.4050 - val_accuracy: 0.5001
Epoch 4/300
200/200 [==============================] - 62s 309ms/step - loss: 1.2474 - accuracy: 0.5541 - val_loss: 1.2844 - val_accuracy: 0.5363
Epoch 5/300
200/200 [==============================] - 61s 307ms/step - loss: 1.1475 - accuracy: 0.5887 - val_loss: 1.5610 - val_accuracy: 0.5167
Epoch 6/300
200/200 [==============================] - 62s 308ms/step - loss: 1.0964 - accuracy: 0.6190 - val_loss: 1.3549 - val_accuracy: 0.5353
Epoch 7/300
200/200 [==============================] - 62s 308ms/step - loss: 1.0229 - accuracy: 0.6393 - val_loss: 1.1992 - val_accuracy: 0.5914
Epoch 8/300
200/200 [==============================] - 62s 308ms/step - loss: 0.9655 - accuracy: 0.6654 - val_loss: 1.0213 -

Epoch 58/300
200/200 [==============================] - 61s 307ms/step - loss: 0.4059 - accuracy: 0.8659 - val_loss: 0.6407 - val_accuracy: 0.7991
Epoch 59/300
200/200 [==============================] - 61s 306ms/step - loss: 0.4101 - accuracy: 0.8644 - val_loss: 0.4718 - val_accuracy: 0.8484
Epoch 60/300
200/200 [==============================] - 61s 307ms/step - loss: 0.3843 - accuracy: 0.8713 - val_loss: 0.4835 - val_accuracy: 0.8452
Epoch 61/300
200/200 [==============================] - 61s 307ms/step - loss: 0.3694 - accuracy: 0.8774 - val_loss: 0.5543 - val_accuracy: 0.8283
Epoch 62/300
200/200 [==============================] - 61s 307ms/step - loss: 0.4125 - accuracy: 0.8650 - val_loss: 0.4471 - val_accuracy: 0.8567
Epoch 63/300
200/200 [==============================] - 61s 307ms/step - loss: 0.3999 - accuracy: 0.8675 - val_loss: 0.5302 - val_accuracy: 0.8348
Epoch 64/300
200/200 [==============================] - 61s 307ms/step - loss: 0.3782 - accuracy: 0.8773 - val_loss: 0

200/200 [==============================] - 62s 308ms/step - loss: 0.2984 - accuracy: 0.9032 - val_loss: 0.4235 - val_accuracy: 0.8655
Epoch 114/300
200/200 [==============================] - 62s 309ms/step - loss: 0.2743 - accuracy: 0.9097 - val_loss: 0.4459 - val_accuracy: 0.8652
Epoch 115/300
200/200 [==============================] - 61s 307ms/step - loss: 0.2675 - accuracy: 0.9126 - val_loss: 0.4425 - val_accuracy: 0.8611
Epoch 116/300
200/200 [==============================] - 61s 307ms/step - loss: 0.2888 - accuracy: 0.9047 - val_loss: 0.4252 - val_accuracy: 0.8666
Epoch 117/300
200/200 [==============================] - 61s 307ms/step - loss: 0.2830 - accuracy: 0.9071 - val_loss: 0.3977 - val_accuracy: 0.8734
Epoch 118/300
200/200 [==============================] - 62s 308ms/step - loss: 0.2547 - accuracy: 0.9139 - val_loss: 0.4208 - val_accuracy: 0.8741
Epoch 119/300
200/200 [==============================] - 61s 307ms/step - loss: 0.2660 - accuracy: 0.9104 - val_loss: 0.3850 -

200/200 [==============================] - 62s 308ms/step - loss: 0.1417 - accuracy: 0.9566 - val_loss: 0.3176 - val_accuracy: 0.9033
Epoch 169/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1371 - accuracy: 0.9577 - val_loss: 0.3144 - val_accuracy: 0.9025
Epoch 170/300
200/200 [==============================] - 61s 307ms/step - loss: 0.1352 - accuracy: 0.9583 - val_loss: 0.3200 - val_accuracy: 0.9027
Epoch 171/300
200/200 [==============================] - 61s 307ms/step - loss: 0.1441 - accuracy: 0.9553 - val_loss: 0.3129 - val_accuracy: 0.9042
Epoch 172/300
200/200 [==============================] - 61s 306ms/step - loss: 0.1414 - accuracy: 0.9560 - val_loss: 0.3136 - val_accuracy: 0.9063
Epoch 173/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1336 - accuracy: 0.9589 - val_loss: 0.3080 - val_accuracy: 0.9075
Epoch 174/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1317 - accuracy: 0.9584 - val_loss: 0.3110 -

200/200 [==============================] - 62s 309ms/step - loss: 0.1136 - accuracy: 0.9676 - val_loss: 0.3208 - val_accuracy: 0.9063
Epoch 224/300
200/200 [==============================] - 62s 310ms/step - loss: 0.1099 - accuracy: 0.9663 - val_loss: 0.3213 - val_accuracy: 0.9065
Epoch 225/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1171 - accuracy: 0.9641 - val_loss: 0.3235 - val_accuracy: 0.9045
Epoch 226/300
200/200 [==============================] - 61s 307ms/step - loss: 0.1202 - accuracy: 0.9624 - val_loss: 0.3274 - val_accuracy: 0.9042
Epoch 227/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1120 - accuracy: 0.9651 - val_loss: 0.3238 - val_accuracy: 0.9044
Epoch 228/300
200/200 [==============================] - 61s 307ms/step - loss: 0.1084 - accuracy: 0.9668 - val_loss: 0.3194 - val_accuracy: 0.9056
Epoch 229/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1064 - accuracy: 0.9674 - val_loss: 0.3223 -

200/200 [==============================] - 62s 308ms/step - loss: 0.1080 - accuracy: 0.9682 - val_loss: 0.3199 - val_accuracy: 0.9061
Epoch 279/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1040 - accuracy: 0.9695 - val_loss: 0.3163 - val_accuracy: 0.9079
Epoch 280/300
200/200 [==============================] - 62s 308ms/step - loss: 0.1029 - accuracy: 0.9709 - val_loss: 0.3175 - val_accuracy: 0.9081
Epoch 281/300
200/200 [==============================] - 61s 307ms/step - loss: 0.1007 - accuracy: 0.9696 - val_loss: 0.3170 - val_accuracy: 0.9086
Epoch 282/300
200/200 [==============================] - 61s 307ms/step - loss: 0.1028 - accuracy: 0.9692 - val_loss: 0.3212 - val_accuracy: 0.9083
Epoch 283/300
200/200 [==============================] - 62s 308ms/step - loss: 0.0985 - accuracy: 0.9723 - val_loss: 0.3206 - val_accuracy: 0.9068
Epoch 284/300
200/200 [==============================] - 61s 307ms/step - loss: 0.1086 - accuracy: 0.9674 - val_loss: 0.3244 -

In [16]:
scores = model.evaluate(X_test, y_test,verbose = 1)
print("Accuracy = %f" % (100 * scores[1]))

10000/10000 [==============================] - 14s 1ms/step
Accuracy = 90.640002
